In [638]:
import math
import scipy
import numpy as np
from numpy.fft import fft
from scipy.integrate import quad
import scipy.optimize
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl

import pandas as pd
# pd.set_option('display.mpl_style', 'default')
# plt.style.use('ggplot')

%matplotlib inline

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

mpl.rcParams.update({'font.size': 13})

from scipy.stats import norm

# Model Parametrs

In [639]:
S0 = 0.00001
S1 = 200.
K = 80.
r = 0.05
q = 0.08
T = 1.
sigma = 0.2

In [640]:
n = 500
S = scipy.linspace(S0, S1, n)

# Delta
Это количество акций, которыми можно захеджировать опцион

$d_{\pm} = \frac{ln(\frac{S}{K}) + (r \pm \sigma^2/2)(T - t)}{\sigma\sqrt{T - t}}$

In [641]:
def d1(Sp, t):
    return (np.log(Sp / K) + (r + sigma ** 2 / 2) * t) / (sigma * np.sqrt(t)) 

In [642]:
def d1_div(Sp, t):
    return (np.log(Sp / K) + (r - q + sigma ** 2 / 2) * t) / (sigma * np.sqrt(t)) 

In [643]:
def d2(Sp, t):
    return d1(Sp, t) - sigma * np.sqrt(t)

In [644]:
def d2_div(Sp, t):
    return d1_div(Sp, t) - sigma * np.sqrt(t)

In [645]:
def f(x):
    return norm.pdf(x)

In [646]:
def F(x):
    return stats.norm.cdf(x)

In [647]:
def Z(T):
    return np.e**(- r * T)

# Блек-Шоулз

In [ ]:
def C_E(S, T):
    return S * F(d1(S, T)) - K * F(d2(S, T)) * Z(T)


def P_E(S, T):
    return C_E(S, T) - (S - K * Z(T))


def C_E_div(S, T):
    return np.e**(- q * T) * S * F(d1_div(S, T)) - K * F(d2_div(S, T)) * Z(T) 


def P_E_div(S, T):
    return C_E_div(S, T) - (np.e**(- q * T) * S - K * Z(T))

def C_up(S, T):
    return S

def C_low(S, T):
    return [s - K * Z(T) if s >= K * Z(T) else 0 for s in S]

def P_up(S, T):
    return K * Z(T)

def P_low(S, T):
    return [K * Z(T) - s if s <= K * Z(T) else 0 for s in S]

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(121)
plt.title(u'Call опцион')
plt.plot(S, C_E(S, 1.0), color='red', linewidth=2.5, label='q = 0, r = 0.05')
plt.plot(S, C_low(S, 1.0), color='black', linewidth=1.5, label='q = 0, r = 0.05, lower bound')

q = 0.08
plt.plot(S, C_E_div(S, 1.0), color='blue', linewidth=1.5, label='q = 0.08, r = 0.05')

q = 0.03
plt.plot(S, C_E_div(S, 1.0), color='green', linewidth=1.5, label='q = 0.03, r = 0.05')
plt.legend()
plt.axis([0, 130, 0, 60])
plt.xlabel('S')
plt.ylabel('C',rotation=0)
plt.grid(True)

plt.subplot(122)
plt.title(u'Put опцион')
plt.plot(S, P_E(S, 1.0), color='red', linewidth=2.5, label='q = 0, r = 0.05')
plt.plot(S, P_low(S, 1.0), color='black', linewidth=1.5, label='q = 0, r = 0.05, lower bound')

q = 0.08
plt.plot(S, P_E_div(S, 1.0), color='blue', linewidth=1.5, label='q = 0.08, r = 0.05')

q = 0.03
plt.plot(S, P_E_div(S, 1.0), color='green', linewidth=1.5, label='q = 0.03, r = 0.05')

# plt.axvline(x=61.4, ls='--', color='green', label='$x=61.4$')

plt.legend()
plt.axis([0, 130, 0, 100])
plt.xlabel('S')
plt.ylabel('C',rotation=0)
plt.grid(True)
plt.show()

fig.savefig('Sholes.png', dpi=fig.dpi, transparent=True)

In [ ]:
def Delta_call(S, T):
    return F(d1(S, T))


def Delta_call_div(S, T):
    return np.e**(- q * T) * F(d1_div(S, T))

In [ ]:
def Delta_put(S, T):
    return Delta_call(S, T) - 1


def Delta_put_div(S, T):
    return Delta_call_div(S, T) - np.e**( - q * T)

In [ ]:
# n=500j
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# X, Y = np.mgrid[S0:S1:n, 0.0001: 100: n]
# Z = Delta_call(X, Y)
# ax.plot_surface(X, Y, Z, cmap=cm.RdYlBu, lw=1, rstride=1, cstride=1, alpha=0.75)
# # ax.contour(X, Y, Z, 10, lw=3, cmap="autumn_r", linestyles="solid", offset=-1)
# # ax.contour(X, Y, Z, 10, lw=3, colors="k", linestyles="solid")
# plt.show()

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

q = 0.08
plt.subplot(121)
plt.plot(S, Delta_call(S, 0.8), color='red', linewidth=2, label='q = 0, r = 0.05')
plt.plot(S, Delta_call_div(S, 0.8), color='blue', linewidth=2, label='q = 0.08, r = 0.05')
plt.title(u'Call опцион')
plt.legend()
plt.xlabel('Spot')
plt.ylabel('Delta')
plt.grid(True)

plt.subplot(122)
plt.plot(S, Delta_put(S, 0.8), color='red', linewidth=2, label='q = 0, r = 0.05')
plt.plot(S, Delta_put_div(S, 0.8), color='blue', linewidth=2, label='q = 0.08, r = 0.05')
plt.axvline(x=61.4, ls='--', color='green', label='$x=61.4$')
plt.title(u'Put опцион')
plt.legend()
plt.xlabel('Spot')
plt.ylabel('Delta')
plt.grid(True)

# plt.subplot(223)
# plt.plot(S, Delta_call_div(S, 0.8))
# plt.plot(S, Delta_call_div(S, 0.5))
# plt.plot(S, Delta_call_div(S, 0.2))
# plt.title('Call option dividends')
# plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
# plt.xlabel('Spot')
# plt.ylabel('Delta')
# plt.grid(True)

# plt.subplot(224)
# plt.plot(S, Delta_put_div(S, 0.8))
# plt.plot(S, Delta_put_div(S, 0.5))
# plt.plot(S, Delta_put_div(S, 0.2))
# plt.title('Put option dividends')
# plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
# plt.xlabel('Spot')
# plt.ylabel('Delta')
# plt.grid(True)

plt.show()
fig.savefig('Delta.png', dpi=fig.dpi, transparent=True)

10: 48.9

8: 49.6

2: 56.4

0.8: 61.4

0.4: 64.9

In [ ]:
Delta_put(80, 0.8) - Delta_put_div(80, 0.8)

In [ ]:
np.e ** (-0.08 * 0.8) * 80

In [ ]:
def Stock(S_0, T, Z):
    return S_0 * np.e ** ((r - sigma ** 2 / 2) * T + sigma * T ** 0.5 * Z)


def Stock_q(S_0, T, Z):
    return S_0 * np.e ** ((r - q - sigma ** 2 / 2) * T + sigma * T ** 0.5 * Z)

In [ ]:
sigma = 0.2
Z = np.random.normal(0, 1, 10000)
q = 0.08
# plt.hist(Stock_q(61.4, 0.8, Z),bins=50)
plt.hist(Stock(61.4, 0.8, Z), bins=50, color='red')
plt.hist(Stock_q(61.4, 0.8, Z),bins=50)
plt.axvline(x=61.4, ls='--', color='black', label='$x=61.4$')
plt.show()

In [ ]:
print Delta_put(61.4, 0.008)
print Delta_put_div(80, 0.008)

In [ ]:
(80 - 80 * np.e ** ( - 0.05 * 0.8)) * np.e ** (r * 0.8)

In [ ]:
(80 - 80 * np.e ** ( - 0.05 * 0.8))

Узнаем S пересечения дельт путов

In [ ]:
q = 0.08
def func(S):
    return Delta_put(S, 0.8) - Delta_put_div(S, 0.8)

inter = scipy.optimize.fsolve(func, 70)
inter

In [ ]:
K = 80.
r = 0.05
q = 0.04
T = 1.
sigma = 0.2

In [ ]:
n = 500.
ttm = scipy.linspace(0.0001, 100., n)

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(121)
plt.plot(ttm, Delta_call(75, ttm))
plt.plot(ttm, Delta_call(80, ttm))
plt.plot(ttm, Delta_call(85, ttm))
plt.title(u'Call опцион')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
plt.axis([0, 35, 0, 1.])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Delta')
plt.grid(True)

# plt.subplot(122)
# plt.plot(ttm, Delta_put(75, ttm))
# plt.plot(ttm, Delta_put(80, ttm))
# plt.plot(ttm, Delta_put(85, ttm))
# plt.title('Put option')
# plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
# plt.axis([0, 35, -1, 0])
# plt.xlabel('time-to-maturity')
# plt.ylabel('Delta')
# plt.grid(True)

plt.subplot(122)
plt.plot(ttm, Delta_call_div(75, ttm))
plt.plot(ttm, Delta_call_div(80, ttm))
plt.plot(ttm, Delta_call_div(85, ttm))
plt.plot(ttm, np.e ** (- q * ttm))
plt.title(u'Call опцион с дивидендами')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money', '$V_k$'])
plt.axis([0, 35, 0, 1])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Delta')
plt.grid(True)

# plt.subplot(224)
# plt.plot(ttm, Delta_put_div(75, ttm))
# plt.plot(ttm, Delta_put_div(80, ttm))
# plt.plot(ttm, Delta_put_div(85, ttm))
# plt.title('Put option dividends')
# plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
# plt.axis([0, 35, -1, 0])
# plt.xlabel('time-to-maturity')
# plt.ylabel('Delta')
plt.grid(True)

plt.show()

fig.savefig('Delta_ttm.png', dpi=fig.dpi, transparent=True)

ATM: $f(0) = 0.5$

OTM: $f(0) = -inf$

ITM: $f(0) = 1$

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

sigma = 0.02
plt.subplot(121)
plt.plot(ttm, Delta_call(75, ttm))
plt.plot(ttm, Delta_call(80, ttm))
plt.plot(ttm, Delta_call(85, ttm))
plt.title(u'Call опцион')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
plt.axis([0, 10, 0, 1.])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Delta')
plt.grid(True)

sigma = 0.2
plt.subplot(122)
plt.plot(ttm, Delta_call_div(75, ttm))
plt.plot(ttm, Delta_call_div(80, ttm))
plt.plot(ttm, Delta_call_div(85, ttm))
plt.plot(ttm, np.e ** (- q * ttm))
plt.title(u'Call опцион с дивидендами')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money', '$e^{-q * T}$'])
plt.axis([0, 70, 0, 1])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Delta')
plt.grid(True)

plt.show()

In [ ]:
F(-0)

# Gamma

$gamma_{C_{E}} = \frac{1}{S\sigma\sqrt{T-t}} delta_{C_{E}}'$

In [ ]:
def Gamma_call(S, T):
    return f(d1(S, T)) / (S * sigma * np.sqrt(T))



def Gamma_call_div(S, T):
    return np.e**(- q * T) * f(d1_div(S, T)) / (S * sigma * np.sqrt(T))


In [ ]:
K = 80.
r = 0.05
q = 0.08
sigma = 0.2

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(121)
plt.plot(S, Gamma_call(S, 0.8))
plt.plot(S, Gamma_call(S, 0.5))
plt.plot(S, Gamma_call(S, 0.2))
plt.title(u'Call/Put опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Gamma')
plt.axis([50, 100, 0, 0.1])
plt.grid(True)

plt.subplot(122)
plt.plot(S, Gamma_call_div(S, 0.8))
plt.plot(S, Gamma_call_div(S, 0.5))
plt.plot(S, Gamma_call_div(S, 0.2))
plt.title(u'Call/Put опцион с дивидендами')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Gamma')
plt.axis([50, 100, 0, 0.1])
plt.grid(True)

plt.show()
fig.savefig('Gamma.png', dpi=fig.dpi, transparent=True)

In [ ]:
fig=plt.figure(figsize=(8, 8), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(S, Gamma_call(S, 0.8))
plt.plot(S, Gamma_call_div(S, 0.8))
plt.title('Call/Put')
plt.legend(['$q=0, r=0.05$', '$q=0.08, r=0.05$'])
plt.xlabel('Spot')
plt.ylabel('Gamma')
# plt.axvline(x=61.4, ls='--', color='green', label='$x=61.4$')
plt.axis([40, 140, 0, 0.04])
plt.grid(True)

plt.show()

In [ ]:
f(d1(10, 0.8))

In [ ]:
Gamma_call(20, 0.8)

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(121)
plt.plot(ttm, Gamma_call(85, ttm))
plt.plot(ttm, Gamma_call(75, ttm))
plt.plot(ttm, Gamma_call(80, ttm))
plt.legend(['in-the-money', 'out of-the-money', 'at-the-money'])
plt.axis([0, 3, 0, 0.2])
plt.title(u'Call/Put опцион')
plt.xlabel('time-to-maturity, years')
plt.ylabel('Gamma')
plt.grid(True)

plt.subplot(122)
plt.plot(ttm, Gamma_call_div(85, ttm))
plt.plot(ttm, Gamma_call_div(75, ttm))
plt.plot(ttm, Gamma_call_div(80, ttm))
plt.legend(['in-the-money', 'out of-the-money', 'at-the-money'])
plt.axis([0, 3, 0, 0.2])
plt.title(u'Call/Put опцион с дивидендами')
plt.xlabel('time-to-maturity, years')
plt.ylabel('Gamma')
plt.grid(True)

plt.show()
fig.savefig('Gamma_ttm.png', dpi=fig.dpi, transparent=True)

In [ ]:
Gamma_call_div(80, 0.00001)

# Vega

$vega_{C_{E}} = delta_{C_{E}}' S\sqrt{T-t}$

In [ ]:
def Vega_call(S, T):
    return S * f(d1(S, T)) * np.sqrt(T)


def Vega_call_div(S, T):
    return S * np.e**( - q * T) * f(d1_div(S, T)) * np.sqrt(T)

In [ ]:
fig=plt.figure(figsize=(8, 8), dpi= 80, facecolor='w', edgecolor='k')

# plt.subplot(121)
plt.plot(S, Vega_call(S, 0.8))
plt.plot(S, Vega_call(S, 0.5))
plt.plot(S, Vega_call(S, 0.2))
plt.title(u'Call/Put опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Vega')
plt.grid(True)

# plt.subplot(122)
# plt.plot(S, Vega_call_div(S, 0.8))
# plt.plot(S, Vega_call_div(S, 0.5))
# plt.plot(S, Vega_call_div(S, 0.2))
# plt.title('Call/Put-option dividends')
# plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
# plt.xlabel('Spot')
# plt.ylabel('Vega')
# plt.grid(True)

plt.show()
fig.savefig('Vega.png', dpi=fig.dpi)

In [ ]:
fig=plt.figure(figsize=(8, 8), dpi= 80, facecolor='w', edgecolor='k')

# plt.subplot(121)
plt.plot(S, Vega_call(S, 0.8))
plt.plot(S, Vega_call_div(S, 0.8))
plt.title(u'Call/Put опцион')
plt.legend(['$r=0.05, q=0$', '$r=0.05, q=0.08$'])
plt.xlabel('Spot')
plt.ylabel('Vega')
plt.grid(True)

plt.show()
fig.savefig('Vega_compar.png', dpi=fig.dpi, transparent=True)

In [ ]:
fig=plt.figure(figsize=(13, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(121)
plt.plot(ttm, Vega_call(75, ttm))
plt.plot(ttm, Vega_call(80, ttm))
plt.plot(ttm, Vega_call(85, ttm))
plt.title(u'Call/Put опцион')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
plt.axis([0, 100, 0, 80])
plt.xlabel('time-to-maturity')
plt.ylabel('Vega')
plt.grid(True)

plt.subplot(122)
plt.plot(ttm, Vega_call_div(75, ttm))
plt.plot(ttm, Vega_call_div(80, ttm))
plt.plot(ttm, Vega_call_div(85, ttm))
plt.title(u'Call/Put опцион с дивидендами')
plt.legend(['out of-the-money', 'at-the-money', 'in-the-money'])
plt.axis([0, 100, 0, 80])
plt.xlabel('time-to-maturity')
plt.ylabel('Vega')
plt.grid(True)

plt.show()
fig.savefig('Vega_ttm.png', dpi=fig.dpi)

In [ ]:
# GV = G1 / V1

In [ ]:
# plt.plot(V1, G1)
# plt.xlabel('Spot')
# plt.ylabel('Vega')
# plt.grid(True)
# plt.show()

# Volatility

In [ ]:
S0 = 0.00001
S1 = 200
K = 80
r = 0.05
T = 0.8
sigma = 0.2

In [ ]:
n = 500
S = scipy.linspace(S0, S1, n)

In [ ]:
def d1_vol(S, sigma):
    return (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T)) 


def d1_vol_div(S, sigma):
    return (np.log(S / K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T)) 

In [ ]:
def Delta_call_vol(S, sigma):
    return F(d1_vol(S, sigma))


def Delta_call_vol_div(S, sigma):
    return np.e**(- q * T) * F(d1_vol_div(S, sigma))

In [ ]:
fig=plt.figure(figsize=(8, 8), dpi= 80, facecolor='w', edgecolor='k')

# plt.subplot(121)
plt.plot(S, Delta_call_vol(S, 0.1))
plt.plot(S, Delta_call_vol(S, 0.2))
plt.plot(S, Delta_call_vol(S, 0.3))
plt.title(u'Call опцион')
plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
plt.xlabel('Spot')
plt.ylabel('Delta')
plt.grid(True)

# plt.subplot(122)
# plt.plot(S, Delta_call_vol_div(S, 0.1))
# plt.plot(S, Delta_call_vol_div(S, 0.2))
# plt.plot(S, Delta_call_vol_div(S, 0.3))
# plt.title('Call-option dividends')
# plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
# plt.xlabel('Spot')
# plt.ylabel('Delta')
# plt.grid(True)

plt.show()

fig.savefig('Delta_vol.png', dpi=fig.dpi, transparent=True)

In [ ]:
def Gamma_call_vol(S, sigma):
    return f(d1_vol(S, sigma)) / (S * sigma * np.sqrt(T))

In [ ]:
fig=plt.figure(figsize=(6, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(S, Gamma_call_vol(S, 0.1))
plt.plot(S, Gamma_call_vol(S, 0.2))
plt.plot(S, Gamma_call_vol(S, 0.3))
plt.title('Call-option')
plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
plt.xlabel('Spot')
plt.ylabel('Gamma')
plt.grid(True)
plt.show()

fig.savefig('Gamma_vol.png', dpi=fig.dpi, transparent=True)

In [ ]:
def Vega_call_vol(S, sigma):
    return S * f(d1_vol(S, sigma)) * np.sqrt(T)

In [ ]:
fig=plt.figure(figsize=(6, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.plot(S, Vega_call_vol(S, 0.1))
plt.plot(S, Vega_call_vol(S, 0.2))
plt.plot(S, Vega_call_vol(S, 0.3))
plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
plt.xlabel('Spot')
plt.ylabel('Vega')
plt.grid(True)
plt.show()

fig.savefig('Vega_vol.png', dpi=fig.dpi, transparent=True)

# r = 0 (Надо прояснить)

In [ ]:
# S0 = 0.00001
# S1 = 200
# K = 80
# r = 0.
# T = 1.
# sigma = 0.2

In [ ]:
# n = 500
# S = scipy.linspace(S0, S1, n)

In [ ]:


# G1 = f(d1(S, 0.8)) / (S * sigma * np.sqrt(0.8))
# G2 = f(d1(S, 0.5)) / (S * sigma * np.sqrt(0.5))
# G3 = f(d1(S, 0.2)) / (S * sigma * np.sqrt(0.2))

In [ ]:
# plt.plot(S, G1)
# plt.plot(S, G2)
# plt.plot(S, G3)
# plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
# plt.xlabel('Spot')
# plt.ylabel('Gamma')
# plt.grid(True)
# plt.show()

In [ ]:
# V1 = S * f(d1(S, 0.8)) * np.sqrt(0.8)
# V2 = S * f(d1(S, 0.5)) * np.sqrt(0.5)
# V3 = S * f(d1(S, 0.2)) * np.sqrt(0.2)

In [ ]:
# plt.plot(S, V1)
# plt.plot(S, V2)
# plt.plot(S, V3)
# plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
# plt.xlabel('Spot')
# plt.ylabel('Vega')
# plt.grid(True)
# plt.show()

#### Time value = market price - intrinsic value(Underlying - Strike)


it depends on the implied volatility and stock price

#### Time value = Premium - Intrinsic Value

Time value decreases due to the time decay


# Theta

### $theta_{C_{E}} = -\frac{S\sigma}{2\sqrt{T-t}}N'(d_{+}) - rKe^{-r(T-t)}N(d_{-})$


Theta показывает, как цена опциона меняется в цене при приближении к сроку экспирации при неизменной implied volatility

Для long-позиций theta отрицательная, для short - положительная

Когда мы находимся в позиции long, для получениея большей прибыли необходимо, чтобы росла цента актива, либо же увеличивалась волатильность. Однако все это должно произойти в ограниченный промежуток времени и чем меньше этот промежуток, тем меньше будет стоимость опциона, так как вероятность благоприятных исходов уменьшается.

В позиции short theta положительная, можно продать опцион в момент T1 и в момент времени T2 купить его, чтобы закрыть сделку. Однако extrinsic value опциона будет в T2 уже меньше и на этом можно заработать.

In [ ]:
S0 = 0.00001
S1 = 200.
K = 80.
r = 0.05
q = 0.06
T = 1.
sigma = 0.2
n = 500
S = scipy.linspace(S0, S1, n)
ttm = scipy.linspace(0.01, 1, n)

In [ ]:
# Тета call опциона
def Th_call(T, S):
    return - S * sigma * f(d1(S, T)) / (2 * T ** 0.5) - r * K * np.e ** (- r * T) * F(d2(S, T))

# def Th_call_div(T, S):
#     return - S * sigma * f(d1_div(S, T)) / (2 * T ** 0.5) - (r - q) * K * np.e ** (- (r - q) * T) * F(d2_div(S, T))

def Th_call_div(T, S):
    return - (S * sigma * np.e**(-q * T) / (2 * T**0.5) * f(d1(S, T))) - r * K * np.e**(- r * T) * F(d2_div(S, T)) + q * S * np.e**( - q * T) * F(d1_div(S, T))

In [ ]:
# Тета форварда
def Th_forv(T, S):
    return  - K * r * np.e ** (- r * T)

def Th_forv_div(T, S):
    return S * q * np.e ** (- q * T) - K * r * np.e ** (- r * T)

In [ ]:
# Тета put опциона
def Th_put(T, S):
    return Th_call(T, S) - Th_forv(T, S)

def Th_put_div(T, S):
    return Th_call_div(T, S) - Th_forv_div(T, S)

In [ ]:
fig=plt.figure(figsize=(15, 17), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(321)
plt.plot(S, Th_call(0.8, S))
plt.plot(S, Th_call(0.5, S))
plt.plot(S, Th_call(0.2, S))
plt.title(u'Call опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)
# plt.show()

plt.subplot(322)
plt.plot(S, Th_put(0.8, S))
plt.plot(S, Th_put(0.5, S))
plt.plot(S, Th_put(0.2, S))
plt.title(u'Put опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)

plt.subplot(323)
plt.plot(S, Th_call_div(0.8, S))
plt.plot(S, Th_call_div(0.5, S))
plt.plot(S, Th_call_div(0.2, S))
plt.plot(S, Th_forv_div(0.8, S))
plt.title(u'Call опцион с дивидендами, $r-q=-0.01$')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y', '$V_k$'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)
# plt.show()

plt.subplot(324)
plt.plot(S, Th_put_div(0.8, S))
plt.plot(S, Th_put_div(0.5, S))
plt.plot(S, Th_put_div(0.2, S))
plt.plot(S, - Th_forv_div(0.8, S))
plt.title(u'Put опцион с дивидендами, $r-q=-0.01$')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y', '$V_k$'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)

q = 0.04

plt.subplot(325)
plt.plot(S, Th_call_div(0.8, S))
plt.plot(S, Th_call_div(0.5, S))
plt.plot(S, Th_call_div(0.2, S))
plt.plot(S, Th_forv_div(0.8, S))
plt.title(u'Call опцион с дивидендами, $r-q=0.01$')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y', '$V_k$'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)
# plt.show()

plt.subplot(326)
plt.plot(S, Th_put_div(0.8, S))
plt.plot(S, Th_put_div(0.5, S))
plt.plot(S, Th_put_div(0.2, S))
plt.plot(S, - Th_forv_div(0.8, S))
plt.title(u'Put опцион с дивидендами, $r-q=0.01$')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y', '$V_k$'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)

fig.tight_layout()
plt.show()
fig.savefig('Theta.png', dpi=fig.dpi, transparent=True)

На графике выше можно видеть, что theta минимальна в at-the-money, так как в таком случае цена опциона состоит целиком из time value, которое в свою очередь зависит от implied volatility и цены активы


In [ ]:
q = 0.06

In [ ]:
fig=plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(221)
plt.plot(ttm, Th_call(ttm, 85))
plt.plot(ttm, Th_call(ttm, 80))
plt.plot(ttm, Th_call(ttm, 75))
plt.title(u'Call опцион')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Theta')
plt.grid(True)

plt.subplot(222)
plt.plot(ttm, Th_put(ttm, 85))
plt.plot(ttm, Th_put(ttm, 80))
plt.plot(ttm, Th_put(ttm, 75))
plt.title(u'Put опцион')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Theta')
plt.grid(True)

plt.subplot(223)
plt.plot(ttm, Th_call_div(ttm, 85))
plt.plot(ttm, Th_call_div(ttm, 80))
plt.plot(ttm, Th_call_div(ttm, 75))
plt.title(u'Call опцион с дивидендами')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Theta')
plt.grid(True)

plt.subplot(224)
plt.plot(ttm, Th_put_div(ttm, 85))
plt.plot(ttm, Th_put_div(ttm, 80))
plt.plot(ttm, Th_put_div(ttm, 75))
plt.title(u'Put опцион с дивидендами')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Theta')
plt.grid(True)

fig.tight_layout()
plt.show()
fig.savefig('Theta_ttm.png', dpi=fig.dpi, transparent=True)

На графике выше можно видеть, что at-the-money option будет терять деньги вплоть до последнего, так как всегда будет оставаться неопределенность с возможностью осуществления благоприятных условий на рынке.
    В то же время эта неопределнность будет ниже для OTM и ITM опционов и при приближении к экспирации падение цены замедлится.

ATM: $f(0) = - inf$

ITM: $f(0) = -4$

OTM: $f(0) = 0$

Forvard: $f(0) = -4$


In [ ]:
Th_call(0.00000001, 85) / 0.00000001

prev = Th_call(0.1, 85)
eps = 0.00000000001
for i in np.arange(0.2, 100000, 0.1):
    y = Th_call(i, 85) / i
    if abs(y - prev) < eps:
        break
    prev = y    
print y  
print i

In [ ]:
Th_call(0.02, 85) / 0.02

In [ ]:
S = scipy.linspace(S0, S1, n)

In [ ]:
fig=plt.figure(figsize=(6, 5), dpi= 80, facecolor='w', edgecolor='k')

sigma=0.1
plt.plot(S, Th_call(0.8, S))
sigma=0.2
plt.plot(S, Th_call(0.8, S))
sigma=0.3
plt.plot(S, Th_call(0.8, S))
plt.title(u'Call опцион')
plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
plt.xlabel('Spot')
plt.ylabel('Theta')
plt.grid(True)
plt.show()

fig.savefig('Theta_vol.png', dpi=fig.dpi, transparent=True)

In [ ]:
sigma = 0.2

# Rho

### $rho_{C_{E}} = (T - t)Ke^{-r(T - t)}N(d_{-})$

In [ ]:
def Rho_call(T, S):
    return T * K * np.e ** (- r * T) * F(d2(S, T))


def Rho_call_div(T, S):
    return T * K * np.e ** (- r * T) * F(d2_div(S, T))

In [ ]:
def Rho_put(T, S):
    return Rho_call(T, S) - K * (T) * np.e**(- r * (T))


def Rho_put_div(T, S):
    return Rho_call_div(T, S) - K * (T) * np.e**(- r * (T))

In [ ]:
fig=plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(221)
plt.plot(S, Rho_call(0.8, S))
plt.plot(S, Rho_call(0.5, S))
plt.plot(S, Rho_call(0.2, S))
plt.title(u'Call опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(222)
plt.plot(S, Rho_put(0.8, S))
plt.plot(S, Rho_put(0.5, S))
plt.plot(S, Rho_put(0.2, S))
plt.title(u'Put опцион')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(223)
plt.plot(S, Rho_call_div(0.8, S))
plt.plot(S, Rho_call_div(0.5, S))
plt.plot(S, Rho_call_div(0.2, S))
plt.title(u'Call опцион с дивидендами')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(224)
plt.plot(S, Rho_put_div(0.8, S))
plt.plot(S, Rho_put_div(0.5, S))
plt.plot(S, Rho_put_div(0.2, S))
plt.title(u'Put опцион с дивидендами')
plt.legend(['T-t=0.8y', 'T-t=0.5y', 'T-t=0.2y'])
plt.xlabel('Spot')
plt.ylabel('Rho')
plt.grid(True)

fig.tight_layout()
plt.show()
fig.savefig('Rho.png', dpi=fig.dpi, transparent=True)

Асимптота: $f(x) = K(T - t)e^{-r(T - t)}$

In [ ]:
def f1(T):
    return K * T * np.e**(- r * T)

In [ ]:
print Rho_call(0.8, 300)
print 0.8 * K * np.e**(- r * 0.8)

In [ ]:
fig=plt.figure(figsize=(15, 10), dpi= 80, facecolor='w', edgecolor='k')

plt.subplot(221)
plt.plot(ttm, Rho_call(ttm, 85))
plt.plot(ttm, Rho_call(ttm, 80))
plt.plot(ttm, Rho_call(ttm, 75))
plt.title(u'Call опцион')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(222)
plt.plot(ttm, Rho_put(ttm, 85))
plt.plot(ttm, Rho_put(ttm, 80))
plt.plot(ttm, Rho_put(ttm, 75))
plt.title(u'Call опцион')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(223)
plt.plot(ttm, Rho_call_div(ttm, 85))
plt.plot(ttm, Rho_call_div(ttm, 80))
plt.plot(ttm, Rho_call_div(ttm, 75))
plt.title(u'Call опцион с дивидендами')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Rho')
plt.grid(True)

plt.subplot(224)
plt.plot(ttm, Rho_put_div(ttm, 85))
plt.plot(ttm, Rho_put_div(ttm, 80))
plt.plot(ttm, Rho_put_div(ttm, 75))
plt.title(u'Put опцион с дивидендами')
plt.legend(['in-the-money', 'at-the-money', 'out of-the-money'])
plt.xlabel('time-to-maturity, years')
plt.ylabel('Rho')
plt.grid(True)

fig.tight_layout()
plt.show()
fig.savefig('Rho_ttm.png', dpi=fig.dpi, transparent=True)

In [ ]:
fig=plt.figure(figsize=(6, 5), dpi= 80, facecolor='w', edgecolor='k')

sigma=0.1
plt.plot(S, Rho_call(0.8, S))
sigma=0.2
plt.plot(S, Rho_call(0.8, S))
sigma=0.3
plt.plot(S, Rho_call(0.8, S))
plt.title(u'Call опцион')
plt.legend(['sigma=10%', 'sigma=20%', 'sigma=30%'])
plt.xlabel('Spot')
plt.ylabel('Rho')
plt.grid(True)
plt.show()

fig.savefig('Rho_vol.png', dpi=fig.dpi, transparent=True)

In [ ]:
[1, 1] ** 2